In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as func
from pathlib import Path
from collections import defaultdict

spark = (
    SparkSession.builder.master("local").appName("Do It Youself - One").getOrCreate()
)

path = Path().home() / "Documents/PySparkCurso/download/Atividades"

24/04/03 13:58:54 WARN Utils: Your hostname, IdeaPad-Gaming-3-15IHU6 resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
24/04/03 13:58:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 13:58:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def gen_vars(path: Path, session: SparkSession) -> defaultdict[str, DataFrame]:
    vars = defaultdict(
        DataFrame,
    )
    for file in path.iterdir():
        vars[file.name.split(".")[0].lower()] = spark.read.load(
            str(file), header=True, format="parquet", inferSchema=True
        )
    return vars


vars = gen_vars(path, spark)
vars

defaultdict(pyspark.sql.dataframe.DataFrame,
            {'itensvendas': DataFrame[ProdutoID: bigint, VendasID: bigint, Quantidade: bigint, ValorUnitario: double, ValorTotal: double, Desconto: string, TotalComDesconto: double],
             'vendas': DataFrame[VendasID: bigint, VendedorID: bigint, ClienteID: bigint, Data: string, Total: double],
             'clientes': DataFrame[ClienteID: bigint, Cliente: string, Estado: string, Genero: string, Status: string],
             'vendedores': DataFrame[VendedorID: bigint, Vendedor: string],
             'produtos': DataFrame[ProdutoID: bigint, Produto: string, Preco: string]})

In [16]:
spark.sql("CREATE DATABASE vendasvarejo;")

DataFrame[]

In [17]:
spark.sql("SHOW DATABASES;").show()

+------------+
|   namespace|
+------------+
|     default|
|vendasvarejo|
+------------+



In [18]:
spark.sql("USE vendasvarejo;")

DataFrame[]

In [19]:
[df.write.saveAsTable(df_name) for df_name, df in vars.items()]

[None, None, None, None, None]

In [20]:
spark.sql("SHOW TABLES;").show()

+------------+-----------+-----------+
|   namespace|  tableName|isTemporary|
+------------+-----------+-----------+
|vendasvarejo|   clientes|      false|
|vendasvarejo|itensvendas|      false|
|vendasvarejo|   produtos|      false|
|vendasvarejo|     vendas|      false|
|vendasvarejo| vendedores|      false|
+------------+-----------+-----------+



In [35]:
spark.sql("SELECT * FROM clientes;").show()

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
|        6|       Adérito Bahía|    MA|     M|  Silver|
|        7|       Aida Dorneles|    RN|     F|  Silver|
|        8|   Alarico Quinterno|    AC|     M|  Silver|
|        9|    Alberto Cezimbra|    AM|     M|  Silver|
|       10|    Alberto Monsanto|    RN|     M|    Gold|
|       11|       Albino Canela|    AC|     M|  Silver|
|       12|     Alceste Varanda|    RR|     F|  Silver|
|       13|  Alcides Carvalhais|    RO|     M|  Silver|
|       14|        Aldo Martins|    GO|     M|  Silver|
|       15|   Alexandra Tabares|    MG|     F|  

Query that shows nome cliente, data venda, produtos, vendedor,valor do item

In [59]:
spark.sql(
    """
    CREATE OR REPLACE GLOBAL TEMP VIEW
        vendas_clientes_vendedores AS
    SELECT
        clientes.Cliente, vendas.Data, vendedores.Vendedor, vendas.VendasID
    FROM
        vendas
    INNER JOIN
        clientes
    ON
        (vendas.ClienteID = clientes.ClienteID)
    INNER JOIN
        vendedores
    ON
        (vendas.VendedorID = vendedores.VendedorID);
"""
)

spark.sql("SELECT * FROM global_temp.vendas_clientes_vendedores;").show()

+-------------------+---------+----------------+--------+
|            Cliente|     Data|        Vendedor|VendasID|
+-------------------+---------+----------------+--------+
|     Cosme Zambujal| 1/1/2019|    Armando Lago|       1|
|  Gertrudes Hidalgo| 1/1/2020|   Iberê Lacerda|       2|
|   Antão Corte-Real| 2/1/2020|Jéssica Castelão|       3|
|   Antão Corte-Real| 2/1/2019|  Hélio Liberato|       4|
|   Antão Corte-Real| 3/1/2018|  Hélio Liberato|       5|
|   Antão Corte-Real| 4/1/2020|  Hélio Liberato|       6|
|   Antão Corte-Real| 6/1/2019|  Hélio Liberato|       7|
|  Gertrudes Infante| 6/1/2019|  Hélio Liberato|       8|
|     Cosme Zambujal| 6/1/2020|Jéssica Castelão|       9|
| Greice Lameirinhas| 6/1/2020|Capitolino Bahía|      10|
|     Brígida Gusmão| 8/1/2019|Jéssica Castelão|      11|
|     Brígida Gusmão| 9/1/2018|Jéssica Castelão|      12|
|     Brígida Gusmão|10/1/2019|Jéssica Castelão|      13|
|     Joaquim Mieiro|12/1/2020|   Daniel Pirajá|      14|
|     Joaquim 

In [79]:
spark.sql(
    """
    CREATE OR REPLACE GLOBAL TEMP VIEW
        vendas_itensvendas_produtos AS
    SELECT
        itensvendas.VendasID, itensvendas.ValorTotal, produtos.Produto
    FROM
        itensvendas
    INNER JOIN
        produtos
    ON
        (itensvendas.ProdutoID = produtos.ProdutoID);
"""
)

spark.sql("SELECT * FROM global_temp.vendas_itensvendas_produtos;").show()

+--------+----------+--------------------+
|VendasID|ValorTotal|             Produto|
+--------+----------+--------------------+
|     400|   18402.0|Bicicleta Altools...|
|     385|   18402.0|Bicicleta Altools...|
|     395|   13784.4|Bicicleta Trinc C...|
|     367|   13018.6|Bicicleta Trinc C...|
|     380|  14077.54|Bicicleta Altools...|
|     346|   16561.8|Bicicleta Altools...|
|     339|   16561.8|Bicicleta Altools...|
|     397|    9201.0|Bicicleta Altools...|
|     346|   15933.6|Bicicleta Aro 29 ...|
|     264|   16561.8|Bicicleta Altools...|
|     355|  11716.74|Bicicleta Trinc C...|
|     376|    8280.9|Bicicleta Altools...|
|     374|    9201.0|Bicicleta Altools...|
|     397|    7524.2|Bicicleta Aro 29 ...|
|     303|   16561.8|Bicicleta Altools...|
|     358|   13018.6|Bicicleta Trinc C...|
|     374|    7658.0|Bicicleta Trinc C...|
|     336|    8510.0|Bicicleta Gts Adv...|
|     399|   6771.78|Bicicleta Aro 29 ...|
|     292|  13543.56|Bicicleta Aro 29 ...|
+--------+-

In [90]:
spark.sql(
    """
    SELECT
        Cliente, Data, Vendedor, ValorTotal, Produto
    FROM 
        global_temp.vendas_clientes_vendedores AS vcv
    INNER JOIN
        global_temp.vendas_itensvendas_produtos AS vip
    ON
        (vcv.VendasID = vip.VendasID);
"""
).show()

+-----------------+--------+----------------+----------+--------------------+
|          Cliente|    Data|        Vendedor|ValorTotal|             Produto|
+-----------------+--------+----------------+----------+--------------------+
|   Cosme Zambujal|1/1/2019|    Armando Lago|   7820.85|Bicicleta Altools...|
|   Cosme Zambujal|1/1/2019|    Armando Lago|     97.75|Bermuda Predactor...|
|   Cosme Zambujal|1/1/2019|    Armando Lago|     135.0|Camiseta Predacto...|
|Gertrudes Hidalgo|1/1/2020|   Iberê Lacerda|     150.4|Luva De Ciclismo ...|
| Antão Corte-Real|2/1/2020|Jéssica Castelão|     155.0|Capacete Gometws ...|
| Antão Corte-Real|2/1/2020|Jéssica Castelão|    5932.0|Bicicleta Gometws...|
| Antão Corte-Real|2/1/2019|  Hélio Liberato|   7820.85|Bicicleta Altools...|
| Antão Corte-Real|2/1/2019|  Hélio Liberato|     97.75|Bermuda Predactor...|
| Antão Corte-Real|2/1/2019|  Hélio Liberato|    5910.0|Bicicleta Gometws...|
| Antão Corte-Real|3/1/2018|  Hélio Liberato|    2955.0|Biciclet